In [1]:
import numpy as np
from datetime import datetime, timedelta

In [2]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity

# And the clock starts
start_time = datetime.now().replace(microsecond=0)
print(start_time)

2023-08-20 17:43:58


In [3]:
np.random.seed(1991)

In [4]:
q_x = 0.05
q_y = 0.05
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])


In [5]:
timesteps = [start_time]
truth = GroundTruthPath([GroundTruthState([0,1,0,1], timestamp=timesteps[0])])
num_steps = 20
for k in range(1, num_steps+1):
    timesteps.append(start_time+timedelta(seconds=k)) # add next time step to list of timesteps
    truth.append(GroundTruthState(
        transition_model.function(truth[k-1], noise=True, time_interval=timedelta(seconds=1)),
        timestamp=timesteps[k]))
        

In [9]:
from stonesoup.plotter import AnimatedPlotterly
plotter = AnimatedPlotterly(timesteps, tail_length = 1)
plotter.plot_ground_truths(truth, [0,2])
plotter.fig

In [13]:
transition_model.matrix(time_interval=timedelta(seconds=1))

array([[1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.]])

In [14]:
transition_model.covar(time_interval=timedelta(seconds=1))

array([[0.01666667, 0.025     , 0.        , 0.        ],
       [0.025     , 0.05      , 0.        , 0.        ],
       [0.        , 0.        , 0.01666667, 0.025     ],
       [0.        , 0.        , 0.025     , 0.05      ]])

In [15]:
from stonesoup.types.detection import Detection
from stonesoup.models.measurement.linear import LinearGaussian

In [29]:
measurement_model = LinearGaussian(
    ndim_state = 4, # num state dimensions (position and velocity in 2D)
    mapping = (0,2), # mapping measurement vector index to state index
    noise_covar = np.array([[10,0], 
                            [0,10]]) 
)


In [30]:
measurements = []
for state in truth:
    measurement = measurement_model.function(state, noise=True)
    measurements.append(Detection(measurement,
                                  timestamp = state.timestamp,
                                  measurement_model=measurement_model))

In [31]:
plotter.plot_measurements(measurements, [0,2])
plotter.fig

## Build a tracker: Kalman filter

In [32]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

In [33]:
from stonesoup.types.state import GaussianState
prior = GaussianState([[0],[1],[0],[1]], np.diag([1.5,0.5,1.5,0.5]), timestamp = start_time)

In [35]:
from stonesoup.types.hypothesis import SingleHypothesis

In [36]:
from stonesoup.types.track import Track
track = Track()
for measurement in measurements:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement)
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

In [37]:
plotter.plot_tracks(track, [0,2], uncertainty=True)
plotter.fig